In [1]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
import xml.etree.ElementTree as ET

df_board_game_rankings = pd.read_csv('.\\.venv\\data\\boardgames_ranks.csv')
df_board_game_rankings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158945 entries, 0 to 158944
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   158945 non-null  int64  
 1   name                 158945 non-null  object 
 2   yearpublished        158945 non-null  int64  
 3   rank                 158945 non-null  int64  
 4   bayesaverage         158945 non-null  float64
 5   average              158945 non-null  float64
 6   usersrated           158945 non-null  int64  
 7   is_expansion         158945 non-null  int64  
 8   abstracts_rank       1424 non-null    float64
 9   cgs_rank             362 non-null     float64
 10  childrensgames_rank  1073 non-null    float64
 11  familygames_rank     3314 non-null    float64
 12  partygames_rank      912 non-null     float64
 13  strategygames_rank   2992 non-null    float64
 14  thematic_rank        1671 non-null    float64
 15  wargames_rank    

Filter out games with rank of 0

In [2]:
df_bgr2 = pd.read_csv('.\\.venv\\data\\boardgames_ranks.csv')
zerorank = df_bgr2[(df_bgr2['rank'] == 0)].index
df_bgr_nozero = pd.DataFrame(df_bgr2.drop(zerorank, inplace = True))
df_bgr2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27288 entries, 0 to 27287
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   27288 non-null  int64  
 1   name                 27288 non-null  object 
 2   yearpublished        27288 non-null  int64  
 3   rank                 27288 non-null  int64  
 4   bayesaverage         27288 non-null  float64
 5   average              27288 non-null  float64
 6   usersrated           27288 non-null  int64  
 7   is_expansion         27288 non-null  int64  
 8   abstracts_rank       1424 non-null   float64
 9   cgs_rank             362 non-null    float64
 10  childrensgames_rank  1073 non-null   float64
 11  familygames_rank     3314 non-null   float64
 12  partygames_rank      912 non-null    float64
 13  strategygames_rank   2992 non-null   float64
 14  thematic_rank        1671 non-null   float64
 15  wargames_rank        4311 non-null  

Grabbing top 300 overall rank from boardgames_ranks.csv

In [3]:
df_bgr2.loc[(df_bgr2['rank'] >=1) & (df_bgr2['rank'] <=300)]

,id,name,yearpublished,rank,bayesaverage,average,usersrated,is_expansion,abstracts_rank,cgs_rank,childrensgames_rank,familygames_rank,partygames_rank,strategygames_rank,thematic_rank,wargames_rank
0,224517,Brass: Birmingham,2018,1,8.41277,8.59213,48001,0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1,161936,Pandemic Legacy: Season 1,2015,2,8.37571,8.52498,54148,0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN
2,174430,Gloomhaven,2017,3,8.34398,8.57972,63128,0,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN
3,342942,Ark Nova,2021,4,8.33959,8.53505,46382,0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
4,233078,Twilight Imperium: Fourth Edition,2017,5,8.23662,8.59240,24556,0,NaN,NaN,NaN,NaN,NaN,5.0,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,27833,Steam,2009,296,7.19002,7.55062,10278,0,NaN,NaN,NaN,NaN,NaN,200.0,NaN,NaN
296,291572,Oath,2021,297,7.18841,7.79008,7749,0,NaN,NaN,NaN,NaN,NaN,210.0,80.0,NaN
297,204583,Kingdomino,2016,298,7.18824,7.30485,48576,0,NaN,NaN,NaN,68.0,NaN,NaN,NaN,NaN
298,127060,Bora Bora,2013,299,7.18781,7.55639,10326,0,NaN,NaN,NaN,NaN,NaN,203.0,NaN,NaN


Replacing NaN with '0'

In [5]:
df_bgr2.fillna('0', inplace=True)
df_bgr2.head(10)

,id,name,yearpublished,rank,bayesaverage,average,usersrated,is_expansion,abstracts_rank,cgs_rank,childrensgames_rank,familygames_rank,partygames_rank,strategygames_rank,thematic_rank,wargames_rank
0,224517,Brass: Birmingham,2018,1,8.41277,8.59213,48001,0,0,0,0,0,0,1.0,0,0
1,161936,Pandemic Legacy: Season 1,2015,2,8.37571,8.52498,54148,0,0,0,0,0,0,2.0,1.0,0
2,174430,Gloomhaven,2017,3,8.34398,8.57972,63128,0,0,0,0,0,0,4.0,2.0,0
3,342942,Ark Nova,2021,4,8.33959,8.53505,46382,0,0,0,0,0,0,3.0,0,0
4,233078,Twilight Imperium: Fourth Edition,2017,5,8.23662,8.59240,24556,0,0,0,0,0,0,5.0,3.0,0
5,316554,Dune: Imperium,2020,6,8.23137,8.43403,47922,0,0,0,0,0,0,6.0,0,0
6,167791,Terraforming Mars,2016,7,8.20734,8.35483,101433,0,0,0,0,0,0,7.0,0,0
7,115746,War of the Ring: Second Edition,2011,8,8.19064,8.54570,22008,0,0,0,0,0,0,0,4.0,1.0
8,187645,Star Wars: Rebellion,2016,9,8.16964,8.41835,33163,0,0,0,0,0,0,0,5.0,0
9,291457,Gloomhaven: Jaws of the Lion,2020,10,8.15191,8.42294,35486,0,0,0,0,0,0,9.0,7.0,0


Converting rank fields to int.

In [6]:
convert_dict = {'abstracts_rank': int,
                'cgs_rank': int,
                'childrensgames_rank': int,
                'familygames_rank': int,
                'partygames_rank': int,
                'strategygames_rank': int,
                'thematic_rank': int,
                'wargames_rank': int}
df_bgr2 = df_bgr2.astype(convert_dict)
print(df_bgr2.dtypes)

id                       int64
name                    object
yearpublished            int64
rank                     int64
bayesaverage           float64
average                float64
usersrated               int64
is_expansion             int64
abstracts_rank           int64
cgs_rank                 int64
childrensgames_rank      int64
familygames_rank         int64
partygames_rank          int64
strategygames_rank       int64
thematic_rank            int64
wargames_rank            int64
dtype: object


In [7]:
df_bgr2.head(20)

,id,name,yearpublished,rank,bayesaverage,average,usersrated,is_expansion,abstracts_rank,cgs_rank,childrensgames_rank,familygames_rank,partygames_rank,strategygames_rank,thematic_rank,wargames_rank
0,224517,Brass: Birmingham,2018,1,8.41277,8.59213,48001,0,0,0,0,0,0,1,0,0
1,161936,Pandemic Legacy: Season 1,2015,2,8.37571,8.52498,54148,0,0,0,0,0,0,2,1,0
2,174430,Gloomhaven,2017,3,8.34398,8.57972,63128,0,0,0,0,0,0,4,2,0
3,342942,Ark Nova,2021,4,8.33959,8.53505,46382,0,0,0,0,0,0,3,0,0
4,233078,Twilight Imperium: Fourth Edition,2017,5,8.23662,8.59240,24556,0,0,0,0,0,0,5,3,0
5,316554,Dune: Imperium,2020,6,8.23137,8.43403,47922,0,0,0,0,0,0,6,0,0
6,167791,Terraforming Mars,2016,7,8.20734,8.35483,101433,0,0,0,0,0,0,7,0,0
7,115746,War of the Ring: Second Edition,2011,8,8.19064,8.54570,22008,0,0,0,0,0,0,0,4,1
8,187645,Star Wars: Rebellion,2016,9,8.16964,8.41835,33163,0,0,0,0,0,0,0,5,0
9,291457,Gloomhaven: Jaws of the Lion,2020,10,8.15191,8.42294,35486,0,0,0,0,0,0,9,7,0


Rounding averages to 2 digits.

In [8]:
df_bgr2.bayesaverage = df_bgr2.bayesaverage.round(2)
df_bgr2.average = df_bgr2.average.round(2)
df_bgr2.head(50)

,id,name,yearpublished,rank,bayesaverage,average,usersrated,is_expansion,abstracts_rank,cgs_rank,childrensgames_rank,familygames_rank,partygames_rank,strategygames_rank,thematic_rank,wargames_rank
0,224517,Brass: Birmingham,2018,1,8.41,8.59,48001,0,0,0,0,0,0,1,0,0
1,161936,Pandemic Legacy: Season 1,2015,2,8.38,8.52,54148,0,0,0,0,0,0,2,1,0
2,174430,Gloomhaven,2017,3,8.34,8.58,63128,0,0,0,0,0,0,4,2,0
3,342942,Ark Nova,2021,4,8.34,8.54,46382,0,0,0,0,0,0,3,0,0
4,233078,Twilight Imperium: Fourth Edition,2017,5,8.24,8.59,24556,0,0,0,0,0,0,5,3,0
5,316554,Dune: Imperium,2020,6,8.23,8.43,47922,0,0,0,0,0,0,6,0,0
6,167791,Terraforming Mars,2016,7,8.21,8.35,101433,0,0,0,0,0,0,7,0,0
7,115746,War of the Ring: Second Edition,2011,8,8.19,8.55,22008,0,0,0,0,0,0,0,4,1
8,187645,Star Wars: Rebellion,2016,9,8.17,8.42,33163,0,0,0,0,0,0,0,5,0
9,291457,Gloomhaven: Jaws of the Lion,2020,10,8.15,8.42,35486,0,0,0,0,0,0,9,7,0


In [ ]:
tr